In [3]:
import pandas as pd

In [4]:
raw_df = pd.read_csv('../data/entities/Q884.csv')
[_ for _ in raw_df.columns if 'value' in _]

['business.value',
 'businessLabel.value',
 'origindate.value',
 'country.value',
 'industries.value',
 'ceos.value',
 'chairs.value',
 'hqs.value',
 'groups.value',
 'officialname.value',
 'employees.value',
 'profit.value',
 'assets.value']

In [5]:
new_names = {'business.value':'id',
'businessLabel.value':'name',
'origindate.value':'start_date',
'country.value':'country_vals',
'industries.value':'industry_vals',
'ceos.value':'ceo_vals',
'chairs.value':'chair_vals',
'hqs.value':'hq_vals',
'groups.value':'group_vals',
'employees.value':'employee_count',#need to make bands for these
'profit.value':'profit',
'assets.value':'assets', 
'equity.value':'equity',
'markcap.value':'market_cap'}

In [7]:
df = pd.DataFrame(columns=list(new_names))
prev_cols = [_ for _ in list(new_names) if _ in raw_df.columns]
df[prev_cols] = raw_df[prev_cols]
df.columns = df.columns.map(new_names)
df['id'] = df['id'].str.replace('http://www.wikidata.org/entity/','')
#df['country'] = df['country'].str.replace('http://www.wikidata.org/entity/','')
val_cols = [_ for _ in df.columns if '_vals' in _]
ent_df = df.drop(val_cols,axis=1)
ent_df = ent_df.fillna('')
ent_df

,id,name,start_date,employee_count,profit,assets,equity,market_cap
0,Q495027,Busan IPark,1983-01-01T00:00:00Z,,,,,
1,Q17684312,Kakao,1995-02-16T00:00:00Z,,,,,
2,Q50596,Woollim Entertainment,2003-01-01T00:00:00Z,,,,,
3,Q563107,Hanwha Aerospace,1977-01-01T00:00:00Z,,,,,
4,Q4982682,Bucheon FC 1995,2007-01-01T00:00:00Z,,,,,
...,...,...,...,...,...,...,...,...
281,Q484872,Jin Air,2008-01-01T00:00:00Z,,,,,
282,Q494369,Woori Bank,1899-01-01T00:00:00Z,,,,,
283,Q82299720,Asian Film Academy,,,,,,
284,Q12599305,Brand New Music,2011-01-01T00:00:00Z,,,,,


In [10]:
mult_df = df[val_cols+['id']]
redex = mult_df.copy()
redex.index = mult_df['id']
redex['country_vals'] = redex['country_vals'].str.replace('http://www.wikidata.org/entity/','')

In [11]:
def dic_to_ls(dic,typ):
    ret_ls = []
    for k, vs in dic.items():
        for v in vs:
            ret_ls.append({'id':k,'val_name':v,'typ':typ})
    return ret_ls

In [17]:
ind_ls = dic_to_ls(redex['industry_vals'].dropna().str.split(',').to_dict(), 'industry')
ceo_ls = dic_to_ls(redex['ceo_vals'].dropna().str.split(',').to_dict(), 'ceo')
chair_ls = dic_to_ls(redex['chair_vals'].dropna().str.split(',').to_dict(), 'chairman')
hq_ls = dic_to_ls(redex['hq_vals'].dropna().str.split(',').to_dict(), 'headquarters')
group_ls = dic_to_ls(redex['group_vals'].dropna().str.split(',').to_dict(),'grouped_in')
country_ls = dic_to_ls(redex['country_vals'].dropna().str.split(',').to_dict(),'resides_in')
ent_edge_df = pd.DataFrame(ind_ls+ceo_ls+chair_ls+hq_ls+group_ls+country_ls, columns = ['id','val_name','typ'])

In [18]:
ent_edge_df

,id,val_name,typ
0,Q17684312,Internet,industry
1,Q50596,music industry,industry
2,Q563107,aerospace,industry
3,Q483938,phonographic industry,industry
4,Q483938,music,industry
...,...,...,...
510,Q484054,Q884,resides_in
511,Q494369,Q884,resides_in
512,Q82299720,Q884,resides_in
513,Q12599305,Q884,resides_in


In [12]:
raw_df = pd.read_csv('../data/edges/Q884.csv')
[_ for _ in raw_df.columns if 'value' in _]

['company.value',
 'owner.value',
 'companyLabel.value',
 'ownerLabel.value',
 'acquiredate.value']

In [13]:
new_names = {'company.value':'owned_id',
'companyLabel.value':'owned_name',
'owner.value':'owner_id',
'ownerLabel.value':'owner_name',
'acquiredate.value':'date'}

In [14]:
df = pd.DataFrame(columns=list(new_names))
prev_cols = [_ for _ in list(new_names) if _ in raw_df.columns]
df[prev_cols] = raw_df[prev_cols]
df.columns = df.columns.map(new_names)
df['owned_id'] = df['owned_id'].str.replace('http://www.wikidata.org/entity/','')
df['owner_id'] = df['owner_id'].str.replace('http://www.wikidata.org/entity/','')
edge_df = df.copy()
edge_df = edge_df.fillna('')

In [16]:
def get_countries(directory):
    countries = os.listdir()
    print(countries)